## Introduction 

Táto lekcia pokryje: 
- Čo je volanie funkcie a jeho použitia 
- Ako vytvoriť volanie funkcie pomocou OpenAI 
- Ako integrovať volanie funkcie do aplikácie 

## Learning Goals 

Po dokončení tejto lekcie budete vedieť a rozumieť: 

- Účelu používania volania funkcie 
- Nastaveniu volania funkcie pomocou služby OpenAI 
- Navrhovaniu efektívnych volaní funkcií pre použitie vo vašej aplikácii


## Pochopenie volaní funkcií

Pre túto lekciu chceme vytvoriť funkciu pre náš vzdelávací startup, ktorá umožní používateľom použiť chatbot na vyhľadávanie technických kurzov. Odporučíme kurzy, ktoré zodpovedajú ich úrovni zručností, aktuálnej pozícii a technológii, o ktorú majú záujem.

Na dokončenie použijeme kombináciu:
 - `OpenAI` na vytvorenie chatovacieho zážitku pre používateľa
 - `Microsoft Learn Catalog API` na pomoc používateľom nájsť kurzy na základe požiadavky používateľa
 - `Function Calling` na spracovanie používateľovho dopytu a jeho odoslanie do funkcie na vykonanie API požiadavky.

Na začiatok sa pozrime, prečo by sme chceli vôbec použiť volanie funkcií:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # získať novú odpoveď od GPT, kde môže vidieť odpoveď funkcie


print(second_response.choices[0].message)


### Prečo volanie funkcií

Ak ste dokončili akúkoľvek inú lekciu v tomto kurze, pravdepodobne rozumiete sile používania veľkých jazykových modelov (LLM). Dúfajme, že vidíte aj niektoré ich obmedzenia.

Volanie funkcií je funkcia služby OpenAI navrhnutá na riešenie nasledujúcich výziev:

Nekonzistentné formátovanie odpovedí:
- Pred volaním funkcií boli odpovede veľkého jazykového modelu neštruktúrované a nekonzistentné. Vývojári museli písať zložitý validačný kód na spracovanie každej varianty výstupu.

Obmedzená integrácia s externými dátami:
- Pred touto funkciou bolo ťažké začleniť dáta z iných častí aplikácie do kontextu chatu.

Štandardizáciou formátov odpovedí a umožnením bezproblémovej integrácie s externými dátami volanie funkcií zjednodušuje vývoj a znižuje potrebu dodatočnej validačnej logiky.

Používatelia nemohli získať odpovede ako „Aké je aktuálne počasie v Štokholme?“. Je to preto, že modely boli obmedzené na čas, kedy boli dáta trénované.

Pozrime sa na príklad nižšie, ktorý ilustruje tento problém:

Povedzme, že chceme vytvoriť databázu študentských údajov, aby sme im mohli navrhnúť správny kurz. Nižšie máme dva popisy študentov, ktoré sú veľmi podobné v údajoch, ktoré obsahujú.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Chceme to poslať do LLM na spracovanie údajov. Toto môže byť neskôr použité v našej aplikácii na odoslanie do API alebo uloženie do databázy.

Vytvorme dva identické prompt-y, v ktorých LLM inštruujeme, aké informácie nás zaujímajú:


Chceme to poslať LLM, aby analyzoval časti, ktoré sú dôležité pre náš produkt. Takže môžeme vytvoriť dva identické prompt-y na inštrukciu LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po vytvorení týchto dvoch promptov ich odošleme do LLM pomocou `openai.ChatCompletion`. Prompt uložíme do premennej `messages` a priradíme rolu `user`. Toto má napodobniť správu od používateľa, ktorá je napísaná chatbotovi.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Teraz môžeme odoslať obe požiadavky na LLM a preskúmať odpoveď, ktorú dostaneme.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Aj keď sú výzvy rovnaké a popisy podobné, môžeme získať rôzne formáty vlastnosti `Grades`.

Ak spustíte vyššie uvedenú bunku viackrát, formát môže byť `3.7` alebo `3.7 GPA`.

Je to preto, že LLM prijíma nestruktúrované údaje vo forme napísanej výzvy a vracia tiež nestruktúrované údaje. Potrebujeme mať štruktúrovaný formát, aby sme vedeli, čo očakávať pri ukladaní alebo používaní týchto údajov.

Použitím funkčného volania môžeme zabezpečiť, že dostaneme späť štruktúrované údaje. Pri používaní funkčného volania LLM v skutočnosti nevolá ani nespúšťa žiadne funkcie. Namiesto toho vytvoríme štruktúru, ktorú má LLM dodržiavať vo svojich odpovediach. Potom tieto štruktúrované odpovede použijeme na určenie, ktorú funkciu spustiť v našich aplikáciách.


![Diagram toku volania funkcie](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.sk.png)


Potom môžeme vziať to, čo funkcia vráti, a poslať to späť do LLM. LLM potom odpovie pomocou prirodzeného jazyka, aby zodpovedal používateľovu otázku.


### Prípady použitia volania funkcií

**Volanie externých nástrojov**  
Chatboty sú skvelé na poskytovanie odpovedí na otázky používateľov. Použitím volania funkcií môžu chatboty využiť správy od používateľov na dokončenie určitých úloh. Napríklad študent môže požiadať chatbota: „Pošli e-mail môjmu inštruktorovi, že potrebujem viac pomoci s týmto predmetom“. Toto môže vyvolať volanie funkcie `send_email(to: string, body: string)`

**Vytváranie API alebo databázových dopytov**  
Používatelia môžu nájsť informácie pomocou prirodzeného jazyka, ktorý sa prevedie na formátovaný dopyt alebo API požiadavku. Príkladom môže byť učiteľ, ktorý sa pýta: „Kto sú študenti, ktorí dokončili poslednú úlohu“, čo môže vyvolať volanie funkcie `get_completed(student_name: string, assignment: int, current_status: string)`

**Vytváranie štruktúrovaných dát**  
Používatelia môžu vziať blok textu alebo CSV a použiť LLM na extrahovanie dôležitých informácií z neho. Napríklad študent môže previesť článok z Wikipédie o mierových dohodách na vytvorenie AI kartičiek na učenie. Toto je možné vykonať pomocou funkcie `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Vytvorenie vášho prvého volania funkcie

Proces vytvorenia volania funkcie zahŕňa 3 hlavné kroky:  
1. Zavolanie API Chat Completions so zoznamom vašich funkcií a správou od používateľa  
2. Prečítanie odpovede modelu na vykonanie akcie, t.j. spustenie funkcie alebo API volania  
3. Vykonanie ďalšieho volania API Chat Completions s odpoveďou z vašej funkcie na použitie týchto informácií na vytvorenie odpovede pre používateľa.


![Tok volania funkcie](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.sk.png)


### Prvky volania funkcie

#### Vstup používateľa

Prvým krokom je vytvoriť správu používateľa. Tá môže byť dynamicky priradená získaním hodnoty z textového vstupu alebo môžete priradiť hodnotu priamo tu. Ak pracujete s API Chat Completions prvýkrát, musíme definovať `role` a `content` správy.

`role` môže byť buď `system` (vytváranie pravidiel), `assistant` (model) alebo `user` (koncový používateľ). Pre volanie funkcie priradíme túto hodnotu ako `user` a príklad otázky.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Vytváranie funkcií. 

Ďalej definujeme funkciu a parametre tejto funkcie. Použijeme tu len jednu funkciu nazvanú `search_courses`, ale môžete vytvoriť viacero funkcií.

**Dôležité** : Funkcie sú zahrnuté v systémovej správe pre LLM a budú započítané do množstva dostupných tokenov, ktoré máte k dispozícii. 


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definície** 

Štruktúra definície funkcie má viacero úrovní, z ktorých každá má svoje vlastné vlastnosti. Tu je rozpis vnorené štruktúry:

**Vlastnosti funkcie na najvyššej úrovni:**

`name` -  Názov funkcie, ktorú chceme, aby bola zavolaná. 

`description` - Toto je popis toho, ako funkcia funguje. Tu je dôležité byť konkrétny a jasný. 

`parameters` - Zoznam hodnôt a formát, ktorý chcete, aby model vytvoril vo svojej odpovedi. 

**Vlastnosti objektu parametrov:**

`type` -  Dátový typ objektu parametrov (zvyčajne "object")

`properties` - Zoznam konkrétnych hodnôt, ktoré model použije vo svojej odpovedi. 

**Vlastnosti jednotlivých parametrov:**

`name` - Implicitne definované kľúčom vlastnosti (napr. "role", "product", "level")

`type` - Dátový typ tohto konkrétneho parametra (napr. "string", "number", "boolean") 

`description` - Popis konkrétneho parametra. 

**Voliteľné vlastnosti:**

`required` - Pole uvádzajúce, ktoré parametre sú potrebné na dokončenie volania funkcie.


### Volanie funkcie  
Po definovaní funkcie ju teraz musíme zahrnúť do volania API Chat Completion. Robíme to pridaním `functions` do požiadavky. V tomto prípade `functions=functions`.  

Existuje tiež možnosť nastaviť `function_call` na `auto`. To znamená, že necháme LLM rozhodnúť, ktorá funkcia by mala byť volaná na základe správy používateľa, namiesto toho, aby sme to priraďovali sami.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Teraz sa pozrime na odpoveď a uvidíme, ako je naformátovaná:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Vidíte, že je zavolaná funkcia podľa mena a z používateľskej správy LLM dokázal nájsť údaje, ktoré zodpovedajú argumentom funkcie.


## 3.Integrácia volaní funkcií do aplikácie. 


Po otestovaní formátovanej odpovede z LLM ju teraz môžeme integrovať do aplikácie. 

### Správa toku 

Aby sme to integrovali do našej aplikácie, urobme nasledujúce kroky: 

Najprv vykonajme volanie na služby OpenAI a uložme správu do premennej s názvom `response_message`. 


In [ ]:
response_message = response.choices[0].message

Teraz definujeme funkciu, ktorá zavolá Microsoft Learn API na získanie zoznamu kurzov:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Ako najlepšiu prax potom skontrolujeme, či model chce zavolať funkciu. Následne vytvoríme jednu z dostupných funkcií a priradíme ju k funkcii, ktorá sa volá.  
Potom vezmeme argumenty funkcie a namapujeme ich na argumenty z LLM.

Nakoniec pridáme správu o volaní funkcie a hodnoty, ktoré boli vrátené správou `search_courses`. To poskytuje LLM všetky informácie, ktoré potrebuje na odpoveď používateľovi pomocou prirodzeného jazyka.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Teraz pošleme aktualizovanú správu do LLM, aby sme mohli dostať odpoveď v prirodzenom jazyku namiesto odpovede vo formáte JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

Skvelá práca! Aby ste mohli pokračovať v učení o OpenAI Function Calling, môžete vytvoriť: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Viac parametrov funkcie, ktoré môžu pomôcť študentom nájsť viac kurzov. Dostupné parametre API nájdete tu: 
 - Vytvorte ďalší volanie funkcie, ktoré zoberie viac informácií od študenta, napríklad ich materinský jazyk 
 - Vytvorte spracovanie chýb, keď volanie funkcie a/alebo volanie API nevráti žiadne vhodné kurzy 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zrieknutie sa zodpovednosti**:
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, majte na pamäti, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
